In [2]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv("ChartbustersParticipantsData\Data_Train.csv",infer_datetime_format=True,encoding='utf-8')

In [5]:
df_test = pd.read_csv("ChartbustersParticipantsData\Data_Test.csv",infer_datetime_format=True,encoding='utf-8')

In [7]:
df_test['Views']=None

In [8]:
df.Name = df.Name.str.extract('([0-9a-zA-Z ]*)')
df_test.Name = df_test.Name.str.extract('([0-9a-zA-Z ]*)')

In [11]:
data = pd.concat([df,df_test],axis=0,sort=False,ignore_index=True)
data.index = data.Unique_ID

In [12]:
data.drop(columns=['Unique_ID'],inplace=True)

In [13]:
data.Name = data.Name.str.extract('([0-9a-zA-Z ]*)')

In [14]:
data.nunique()

Name           1172
Genre            21
Country           1
Song_Name     98072
Timestamp     84593
Views         53964
Comments       2379
Likes          9758
Popularity     6259
Followers      1161
dtype: int64

In [15]:
data.drop(columns=['Country','Song_Name'],inplace=True)

In [16]:
data.Views = pd.to_numeric(data.Views)

In [17]:
for cols in ['Likes','Popularity']:
    data[cols] = data[cols].apply(lambda x: x.replace(",",''))

In [18]:
data['Likes']=np.where(data['Likes'].str.isnumeric()==False,pd.to_numeric(data['Likes'].apply(lambda x: x[0:-1]))*1000,data['Likes']).astype(int)

In [19]:
data['Popularity']=np.where(data['Popularity'].str.isnumeric()==False,pd.to_numeric(data['Popularity'].apply(lambda x: x[0:-1]))*1000,data['Popularity']).astype(int)

In [20]:
data['Timestamp']= pd.to_datetime(data['Timestamp'])

In [21]:
latest = data['Timestamp'].max()

In [22]:
data['Timestamp'] = (latest - data['Timestamp'] ).astype('timedelta64[D]')

In [23]:
cat_cols = pd.get_dummies(data.select_dtypes(exclude=np.number))

In [24]:
data.Timestamp.astype(int)

Unique_ID
413890      331
249453      980
681116     1388
387253      626
1428029     890
           ... 
183403      544
1272578    1129
539454      486
1543013     543
415046     1305
Name: Timestamp, Length: 98073, dtype: int32

In [25]:
data['Likes_Per_Followers'] = np.where(data.Followers>0,data.Likes/data.Followers,0)

In [27]:
data['Comments_Per_Followers'] = np.where(data.Followers>0,data.Comments/data.Followers,0)

In [26]:
data['Comments_Per_Likes'] = np.where(data.Likes>0,data.Comments/data.Likes,0)

In [28]:
data['Songs_Cnt'] = data["Name"].map(data["Name"].value_counts())

In [29]:
data['Total_Likes'] = data['Name'].map(data.groupby('Name').Likes.sum())

In [30]:
data['Zero_Likes'] = np.where(data.Likes==0,1,0)

In [31]:
data['Zero_Comments'] = np.where(data.Comments==0,1,0)

In [32]:
data['Zero_Popularity'] = np.where(data.Popularity==0,1,0)

In [33]:
data['Likes_Per_Timestamp'] = np.where(data.Timestamp>0,data.Likes/data.Timestamp,0)

In [34]:
data['LikesnPopularity'] = data['Likes'] * data['Popularity']

In [35]:
data['Likes_Per_Popularity'] =  np.where(data.Popularity>0,data.Likes/data.Popularity,0)

In [36]:
data['Avg_Likes'] = data['Name'].map(data.groupby('Name').Likes.mean())

In [37]:
from sklearn.preprocessing import MinMaxScaler,StandardScaler

In [38]:
cat_cols.shape

(98073, 1193)

In [176]:
drop_cols = ["Name_"+ name for name in df.Name.unique().tolist() if name not in df_test.Name.unique().tolist()]

In [177]:
cat_cols.drop(columns = drop_cols,inplace=True)

In [178]:
X = pd.concat([cat_cols,data.select_dtypes(include=np.number)],axis=1)

In [179]:
testX = X[X['Views'].isna()]

In [180]:
trainX = X[X['Views'].notna()]

In [181]:
trainY = trainX.pop('Views').astype(int)

In [182]:
testX.pop('Views')

Unique_ID
562546    NaN
907584    NaN
213013    NaN
340312    NaN
41854     NaN
           ..
183403    NaN
1272578   NaN
539454    NaN
1543013   NaN
415046    NaN
Name: Views, Length: 19615, dtype: float64

In [44]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso,LassoCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor

In [115]:
train_x, test_x, train_y,test_y = train_test_split(trainX,trainY,test_size=0.2, random_state=5)

In [116]:
print(train_x.shape,train_y.shape,test_x.shape,test_y.shape)

(62766, 1210) (62766,) (15692, 1210) (15692,)


In [117]:
lin = LassoCV(n_alphas=['0.1','1','10','0.01'])

In [118]:
lin = Lasso(alpha=0.1)
lin.fit(train_x,train_y)

C:\Users\hp\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 5.4344192107609944e+16, tolerance: 90868133607135.53
  positive)


Lasso(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

In [249]:
print("Train RMSE:",np.sqrt(mean_squared_error(train_y,lin.predict(train_x))))

Train RMSE: 1583315.3941994915


In [250]:
print("Test RMSE:",np.sqrt(mean_squared_error(test_y,lin.predict(test_x))))

Test RMSE: 2063732.3987288144


In [320]:
train_x.Timestamp.astype(int)

Unique_ID
212750     1299
574389      501
662009      999
962549        1
530696      489
           ... 
220974      676
1321506     216
448409     1877
212546      717
972679       19
Name: Timestamp, Length: 62766, dtype: int32

In [48]:
def model_fit(model,trainx,trainy,testx,testy):
    model.fit(trainx,trainy)
    trainypred = model.predict(trainx)
    print("Train RMSE:",np.sqrt(mean_squared_error(trainy,trainypred)))
    testypred = model.predict(testx)
    print("Test RMSE:",np.sqrt(mean_squared_error(testy,testypred)))
    #return testypred
    

In [49]:
def model_predict(model,trainx,trainy,testx):
    model.fit(trainx,trainy)
    trainypred = model.predict(trainx)
    print("Train RMSE:",np.sqrt(mean_squared_error(trainy,trainypred)))
    testypred = model.predict(testx)
    return testypred
    

In [50]:
def write_results(ytest,file):
    result_df= pd.DataFrame([ytest]).T
    result_df.columns=['Views']
    result_df.index = testX.index
    writer = ExcelWriter(file+'.xlsx')
    result_df.to_excel(writer,index=True)
    writer.save()

In [68]:
for i in range(2,4):
    rfr = RandomForestRegressor(random_state=5,n_estimators=i,max_features=0.6)
    print("i=",i)
    model_fit(rfr,train_x,train_y,test_x,test_y)

i= 2
Train RMSE: 450489.63902796403
Test RMSE: 862888.77835751
i= 3
Train RMSE: 447853.0216863948
Test RMSE: 717556.9304413007


In [62]:
model_fit(rfr,train_x,train_y,test_x,test_y)

Train RMSE: 361618.0188482583
Test RMSE: 755460.8465098124


In [67]:
from pandas import ExcelWriter
ytest = model_predict(rfr,trainX,trainY,testX)
write_results(ytest,"RandomForest3")

In [69]:
feat_imp = rfr.feature_importances_
feat_df = pd.concat([pd.DataFrame(testX.columns),pd.DataFrame(feat_imp)],axis=1)
feat_df.columns=['Column','Importance']
feat_df.sort_values(by=['Importance'],ascending=False)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features=0.6, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=3, n_jobs=None,
                      oob_score=False, random_state=5, verbose=0,
                      warm_start=False)

In [79]:
imp_cols=feat_df.sort_values(by=['Importance'],ascending=False)['Column'].head(41)

In [80]:
for i in range(9,10):
    rfr = RandomForestRegressor(random_state=5,n_estimators=3)
    #imp_cols=feat_df.sort_values(by=['Importance'],ascending=False)['Column'].head(i)
    rfr.fit(train_x[imp_cols],train_y)
    testypred=rfr.predict(test_x)
    print("i={}:,Test RMSE:{}".format(i,np.sqrt(mean_squared_error(test_y,testypred))))
    

i=9:,Test RMSE:705366.6867239755


## DecisionTreeRegressor

In [183]:
from sklearn.tree import DecisionTreeRegressor

In [184]:
for i in range(50,60):
    dtr = DecisionTreeRegressor(random_state=5)
    imp_cols=feat_df.sort_values(by=['Importance'],ascending=False)['Column'].head(i)
    dtr.fit(train_x[imp_cols],train_y)
    testypred=dtr.predict(test_x[imp_cols])
    print("i={}:,Test RMSE:{}".format(i,np.sqrt(mean_squared_error(test_y,testypred))))
    

i=50:,Test RMSE:842367.0451245424
i=51:,Test RMSE:801593.1010017269
i=52:,Test RMSE:801324.5671863026
i=53:,Test RMSE:786728.3081632694
i=54:,Test RMSE:762283.916528692
i=55:,Test RMSE:751762.9073132903
i=56:,Test RMSE:785896.4882740595
i=57:,Test RMSE:800728.7449786953
i=58:,Test RMSE:814520.6866553405
i=59:,Test RMSE:791229.5361713163


In [185]:
dtr = DecisionTreeRegressor(random_state=5)
imp_cols=feat_df.sort_values(by=['Importance'],ascending=False)['Column'].head(55)
ytest = model_predict(dtr,trainX[imp_cols],trainY,testX[imp_cols])
write_results(ytest,"DTree55")

Train RMSE: 380.687606461636


## KNN Regressor

In [98]:
knn = KNeighborsRegressor(n_neighbors=5)

In [106]:
for i in range(15,40,5):
    knn = KNeighborsRegressor(n_neighbors=5)
    imp_cols=feat_df.sort_values(by=['Importance'],ascending=False)['Column'].head(i)
    x_scaled = StandardScaler().fit_transform(X[imp_cols])
    train_x,test_x,train_y,test_y = train_test_split(x_scaled[:trainX.shape[0],:],trainY,test_size=0.2,random_state=5)
    knn.fit(train_x,train_y)
    testypred=knn.predict(test_x)
    print("i={}:,Test RMSE:{}".format(i,np.sqrt(mean_squared_error(test_y,testypred))))

i=15:,Test RMSE:1136225.8601206902
i=20:,Test RMSE:1112716.393456088
i=25:,Test RMSE:1111325.1263528233
i=30:,Test RMSE:1182778.6420312307
i=35:,Test RMSE:1201285.7889943807


In [110]:
knn = KNeighborsRegressor(n_neighbors=5)
imp_cols=feat_df.sort_values(by=['Importance'],ascending=False)['Column'].head(12)
x_scaled = MinMaxScaler().fit_transform(X[imp_cols])
xtrain = x_scaled[:trainX.shape[0],:]
xtest = x_scaled[trainX.shape[0]:,:]
ytestpred =  model_predict(knn,xtrain,trainY,xtest)
predictions = [round(value) for value in ytestpred ]
write_results(predictions,"KNNMinMax12")

Train RMSE: 773719.4039845001


In [71]:
from xgboost import XGBRegressor
xgb = XGBRegressor()

In [340]:

xgb = XGBRegressor(random_state=5,n_estimators=200)
ytest = model_predict(xgb,trainX[imp_cols],trainY,testX[imp_cols])
write_results(ytest,"XGBoost3")

C:\Users\hp\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\hp\Anaconda3\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


[04:06:39] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Train RMSE: 328611.71695205546


In [179]:
for i in range(4,7):
    xgb = XGBRegressor(random_state=5,n_estimators=500,max_depth=i)
    xgb.fit(train_x[imp_cols],train_y)
    testypred=xgb.predict(test_x[imp_cols])
    print("i={}:,Test RMSE:{}".format(i,np.sqrt(mean_squared_error(test_y,testypred))))
    
    

C:\Users\hp\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[01:28:05] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
i=4:,Test RMSE:681832.7582993128


C:\Users\hp\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[01:29:38] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
i=5:,Test RMSE:648465.7880878866


C:\Users\hp\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[01:31:26] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
i=6:,Test RMSE:640780.9182854426


In [111]:
feat_df[feat_df.Importance>0].sort_values(by=['Importance'],ascending=False).head(60)

,Column,Importance
1195,Likes,0.316472
1196,Popularity,0.238975
1199,Comments_Per_Likes,0.226605
1194,Comments,0.147453
1207,LikesnPopularity,0.033857
1208,Likes_Per_Popularity,0.021280
1206,Likes_Per_Timestamp,0.002577
1202,Total_Likes,0.002008
1198,Likes_Per_Followers,0.001972
1193,Timestamp,0.001778


In [121]:
imp_cols=feat_df.sort_values(by=['Importance'],ascending=False)['Column'].head(60)

In [122]:
from sklearn.ensemble import VotingRegressor

In [123]:
dtree = DecisionTreeRegressor(random_state=5,max_features=12)
rfr = RandomForestRegressor(random_state=5,n_estimators=3,max_features=41)
voting = VotingRegressor(estimators=[
('DTree',dtree),('RanFor',rfr)
])

In [124]:
ytest = model_predict(voting,trainX[imp_cols],trainY,testX[imp_cols])
write_results(ytest,"Voting")

Train RMSE: 259290.8072595667


In [127]:
from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor

In [138]:
ada = AdaBoostRegressor(random_state=5,n_estimators=3)

In [139]:
imp_cols=feat_df.sort_values(by=['Importance'],ascending=False)['Column'].head(12)
ytest = model_predict(ada,trainX[imp_cols],trainY,testX[imp_cols])
write_results(ytest,"AdaBoost")

Train RMSE: 985410.9594317797


In [148]:
gbr = GradientBoostingRegressor(random_state=5,max_features=0.1,n_estimators=150)

In [149]:
ytest = model_predict(gbr,trainX[imp_cols],trainY,testX[imp_cols])
write_results(ytest,"GradientBoost")

Train RMSE: 582768.1920734084


In [155]:
for i in [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]:
    dtr = GradientBoostingRegressor(random_state=5,max_features=i,n_estimators=150)
    #imp_cols=feat_df.sort_values(by=['Importance'],ascending=False)['Column'].head(i)
    dtr.fit(train_x,train_y)
    testypred=dtr.predict(test_x)
    print("i={}:,Test RMSE:{}".format(i,np.sqrt(mean_squared_error(test_y,testypred))))

i=0.1:,Test RMSE:1016176.5525098592
i=0.2:,Test RMSE:877884.5501042964
i=0.3:,Test RMSE:863464.6801454258
i=0.4:,Test RMSE:732602.348306347
i=0.5:,Test RMSE:682597.5831010077
i=0.6:,Test RMSE:729052.2634500534
i=0.7:,Test RMSE:701561.3541226087
i=0.8:,Test RMSE:681433.0326316672
i=0.9:,Test RMSE:706113.5087218691
i=1:,Test RMSE:3392530.1894489764


In [169]:
gbr = GradientBoostingRegressor(random_state=5,max_features=0.8, n_estimators=75)
ytest = model_predict(gbr,trainX,trainY,testX)
write_results(ytest,"GBR1")

Train RMSE: 443449.1836877321


In [228]:
from sklearn.preprocessing import PolynomialFeatures

In [229]:
poly = PolynomialFeatures(degree=2)

In [246]:
imp_cols=feat_df.sort_values(by=['Importance'],ascending=False)['Column'].head(20)
trainXpoly = poly.fit_transform(trainX[imp_cols])

In [247]:
testXpoly = poly.transform(testX[imp_cols])

In [234]:
lin = LinearRegression()

In [238]:
ytest = model_predict(rfr,trainXpoly,trainY,testXpoly)
write_results(ytest,"Polynomial_RFR")

Train RMSE: 289047.50823923585


In [241]:
imp_cols=feat_df.sort_values(by=['Importance'],ascending=False)['Column'].head(20)
DT = DecisionTreeRegressor(random_state=5, max_features=20)
DT.fit(trainX[imp_cols],trainY)


DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=20,
                      max_leaf_nodes=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      presort=False, random_state=5, splitter='best')

In [242]:
testy_DT = DT.predict(testX[imp_cols])

In [243]:
rfr = RandomForestRegressor(random_state=5,n_estimators=40)

In [244]:
imp_cols=feat_df.sort_values(by=['Importance'],ascending=False)['Column'].head(60)
rfr.fit(trainX[imp_cols],trainY)
testy_RF = rfr.predict(testX[imp_cols])

In [248]:
gbr= GradientBoostingRegressor(random_state=5)
gbr.fit(trainXpoly,trainY)
testy_GBR = rfr.predict(testXpoly)

ValueError: Number of features of the model must match the input. Model n_features is 60 and input n_features is 231 

In [250]:
ensemble = np.mean([testy_DT
                              ,testy_RF
                             ], axis = 0)

In [251]:
ensemble

array([214042.9875,  30803.2875,  15332.225 , ...,   4142.    ,
       101363.35  ,  21217.4875])

In [253]:
result_df = pd.DataFrame(ensemble)

In [254]:
result_df.columns=['Views']
result_df.index = testX.index

In [257]:
writer = ExcelWriter('MeanTree.xlsx')
result_df.to_excel(writer,index=True)

In [258]:
result_df

,Views
Unique_ID,
562546,214042.9875
907584,30803.2875
213013,15332.2250
340312,32708.9875
41854,8542.1875
...,...
183403,65378.4500
1272578,99054.7500
539454,4142.0000


In [259]:
rfr = RandomForestRegressor(random_state=5,n_estimators=1000,max_features=0.1)
ytest = model_predict(rfr,trainX,trainY,testX)
write_results(ytest,"RFR")

KeyboardInterrupt: 

In [356]:
xgb1 = XGBRegressor()
parameters = {'nthread':[2], #when use hyperthread, xgboost may become slower
              'objective':['reg:linear'],
              'learning_rate': [.03, 0.05, .07], #so called `eta` value
              'max_depth': [5, 6, 7],
              'min_child_weight': [4],
              'silent': [1],
              'subsample': [0.7],
              'colsample_bytree': [0.7],
              'n_estimators': [100,200,500],
              'max'}

xgb_grid = GridSearchCV(xgb1,
                        parameters,
                        cv = 5,
                        n_jobs = 2,
                        verbose=True)

xgb_grid.fit(trainX[imp_cols],
         trainY)

print(xgb_grid.best_score_)
print(xgb_grid.best_params_)

Fitting 5 folds for each of 27 candidates, totalling 135 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed: 16.4min
[Parallel(n_jobs=2)]: Done 135 out of 135 | elapsed: 46.3min finished
C:\Users\hp\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\hp\Anaconda3\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


0.9692657134175081
{'colsample_bytree': 0.7, 'learning_rate': 0.07, 'max_depth': 5, 'min_child_weight': 4, 'n_estimators': 500, 'nthread': 2, 'objective': 'reg:linear', 'silent': 1, 'subsample': 0.7}


In [159]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor
xgb1 = XGBRegressor()

In [160]:
xgbr = XGBRegressor(colsample_bytree= 0.7, learning_rate= 0.07, max_depth=5, min_child_weight= 4, n_estimators= 500, nthread= 2, objective= 'reg:linear', silent= 1, subsample= 0.7, max_features=0.8)

In [161]:
xgbr.fit(train_x,train_y,eval_set=[(test_x,test_y)],early_stopping_rounds=5, eval_metric="rmse",verbose=25)

C:\Users\hp\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[0]	validation_0-rmse:3.99289e+06
Will train until validation_0-rmse hasn't improved in 5 rounds.
[25]	validation_0-rmse:1.13001e+06
[50]	validation_0-rmse:733185
[75]	validation_0-rmse:687753
Stopping. Best iteration:
[74]	validation_0-rmse:687368



XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.7, gamma=0,
             importance_type='gain', learning_rate=0.07, max_delta_step=0,
             max_depth=5, max_features=0.8, min_child_weight=4, missing=None,
             n_estimators=500, n_jobs=1, nthread=2, objective='reg:linear',
             random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=None, silent=1, subsample=0.7, verbosity=1)

In [360]:
testY=xgbr.predict(testX)

In [163]:
xgbr = XGBRegressor(n_estimators=74, max_features=0.8, colsample_bytree= 0.7, learning_rate= 0.07, max_depth=5, min_child_weight= 4, nthread= 2, objective= 'reg:linear', silent= 1, subsample= 0.7)
testy = model_predict(xgbr,trainX,trainY,testX)
# make predictions for test data


Train RMSE: 382029.08042337326


In [165]:
predictions = [round(value) if value>0 else 0 for value in testy ]
write_results(predictions,"XGBRGridCV2.xlsx")

In [175]:
import lightgbm as lgb
lgb_fit_params={"early_stopping_rounds":50, 
            "eval_metric" : 'rmse', 
            "eval_set" : [(test_x,test_y)],
            'eval_names': ['valid'],
            'verbose':100
           }

lgb_params = {'boosting_type': 'gbdt',
 'metric': 'rmse',
 'verbose': 0,
 'bagging_fraction': 0.8,
 'bagging_freq': 1,
 'feature_fraction': 0.8,
 'lambda_l1': 0.01,
 'lambda_l2': 0.01,
 'learning_rate': 0.1,
 'max_bin': 255,
 'max_depth': -1,
 'min_data_in_bin': 1,
 'min_data_in_leaf': 1,
 'num_leaves': 31}
lgb_params

clf_lgb = lgb.LGBMRegressor(n_estimators=1000, **lgb_params, random_state=5, n_jobs=2)
clf_lgb.fit(train_x, train_y, **lgb_fit_params)
clf_lgb.best_iteration_


C:\Users\hp\Anaconda3\lib\site-packages\lightgbm\callback.py:188: UserWarning: Early stopping is not available in dart mode
  warnings.warn('Early stopping is not available in dart mode')


[100]	valid's rmse: 919277
[200]	valid's rmse: 859791
[300]	valid's rmse: 788422
[400]	valid's rmse: 759870
[500]	valid's rmse: 754670
[600]	valid's rmse: 756325
[700]	valid's rmse: 743843
[800]	valid's rmse: 742173
[900]	valid's rmse: 742669
[1000]	valid's rmse: 725650


0

In [393]:
clf_lgb_full = lgb.LGBMRegressor(n_estimators=int(clf_lgb.best_iteration_*1.2), **lgb_params, random_state=5, n_jobs=2)
clf_lgb_full.fit(trainX, trainY)

LGBMRegressor(bagging_fraction=0.8, bagging_freq=1, boosting_type='gbdt',
              class_weight=None, colsample_bytree=1.0, feature_fraction=0.7,
              importance_type='split', lambda_l1=0.01, lambda_l2=0.01,
              learning_rate=0.1, max_bin=255, max_depth=-1, metric='rmse',
              min_child_samples=20, min_child_weight=0.001, min_data_in_bin=1,
              min_data_in_leaf=1, min_split_gain=0.0, n_estimators=508,
              n_jobs=2, num_leaves=31, objective=None, random_state=5,
              reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
              subsample_for_bin=200000, subsample_freq=0, verbose=0)

In [395]:
ytest = model_predict(clf_lgb_full,trainX[imp_cols],trainY,testX[imp_cols])
write_results(ytest,"LightGBMR")

Train RMSE: 72964.70008252536


In [409]:
data.loc[1120309]

Name                            Queen
Genre                            rock
Timestamp                        4140
Views                             NaN
Comments                            0
Likes                               3
Popularity                          0
Followers                       70299
Followers_Per_Likes             23433
Followers_Per_Comments              0
Likes_Per_Comments                  0
Songs_Cnt                         689
Zero_Likes                          0
Zero_Comments                       1
Zero_Popularity                     1
Likes_Per_Timestamp       0.000724638
LikesnPopularity                    0
Likes_Per_Popularity                0
Mean_Popularity               1.51234
Avg_Likes                     230.435
Name: 1120309, dtype: object

In [410]:
data.loc[data.Name=='Queen']

,Name,Genre,Timestamp,Views,Comments,Likes,Popularity,Followers,Followers_Per_Likes,Followers_Per_Comments,Likes_Per_Comments,Songs_Cnt,Zero_Likes,Zero_Comments,Zero_Popularity,Likes_Per_Timestamp,LikesnPopularity,Likes_Per_Popularity,Mean_Popularity,Avg_Likes
Unique_ID,,,,,,,,,,,,,,,,,,,,
1120183,Queen,rock,2977.0,2701.0,0,8,0,70299,8787.375000,0.0,0.0,689,0,1,1,0.002687,0,0.000000,1.512337,230.435414
1120262,Queen,rock,2977.0,720.0,0,3,0,70299,23433.000000,0.0,0.0,689,0,1,1,0.001008,0,0.000000,1.512337,230.435414
1120467,Queen,rock,10282.0,63.0,0,1,0,70299,70299.000000,0.0,0.0,689,0,1,1,0.000097,0,0.000000,1.512337,230.435414
1120113,Queen,rock,2977.0,1053.0,0,11,0,70299,6390.818182,0.0,0.0,689,0,1,1,0.003695,0,0.000000,1.512337,230.435414
1119950,Queen,rock,1516.0,5023.0,0,10,2,70299,7029.900000,0.0,0.0,689,0,1,0,0.006596,20,5.000000,1.512337,230.435414
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1119916,Queen,rock,129.0,NaN,0,255,13,70299,275.682353,0.0,0.0,689,0,1,0,1.976744,3315,19.615385,1.512337,230.435414
1119931,Queen,rock,857.0,NaN,0,23,5,70299,3056.478261,0.0,0.0,689,0,1,0,0.026838,115,4.600000,1.512337,230.435414
1120618,Queen,rock,10282.0,NaN,0,7,0,70299,10042.714286,0.0,0.0,689,0,1,1,0.000681,0,0.000000,1.512337,230.435414


In [411]:
import pandas_profiling
data.shape

In [ ]:
pandas_profiling.ProfileReport(data)

In [186]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [ ]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(trainX, trainY)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
